# **Setup**

* https://www.youtube.com/watch?v=4YGkfAd2iXM
* https://www.kaggle.com/datasets/kazanova/sentiment140

In [10]:
import pandas as pd 
import kaggle 
import os 
from IPython.display import clear_output

# NLP Packages
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# ML Packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# **Data Import**

In [4]:
os.getcwd()

'c:\\Users\\aksha\\OneDrive\\06 Data Science & Analytics\\Github\\Natural-Language-Processing\\06 Twitter Sentiment Analysis'

In [5]:
import os
from zipfile import ZipFile
def unzip_file(file_path, extract_path):
    with ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

unzip_file('Input Data/sentiment140.zip','Input Data')

## **Load Data File**

In [17]:
df_col_names = ['target','ids','date','flag','user','text']

df = pd.read_csv('Input Data/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',
                 header=None,names=df_col_names)

print(df.shape)
df.head()

(1600000, 6)


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## **Get Stop Words**

In [23]:
# Download Stopwords
nltk.download('stopwords')

stopwords_list = stopwords.words('english')
print(stopwords_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Check Missing Values**

In [18]:
df.isna().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

# **Data Exploration**

## **Target Variable**

In [19]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [21]:
# Convert Value 4 to 1
df.replace({'target':{4:1}},inplace=True)

df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

# **Data Processing**

## **Stemming**

In [48]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',repl=' ',string=content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords_list]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [50]:
text = 'The 17-year-old had been released from prison the night before, nearly five months after being arrested in a 4am Israeli military raid on the home.'

stemming(text)

'year old releas prison night nearli five month arrest isra militari raid home'

In [26]:
df.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [51]:
print('Initial Data Shape: ',df.shape)

# Stemming
df['text_stemmed'] = df['text'].apply(stemming)
print('Modified Data Shape: ',df.shape)

Initial Data Shape:  (1600000, 7)
Modified Data Shape:  (1600000, 7)


In [53]:
df.head()

,target,ids,date,flag,user,text,text_stemmed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


# **Model Implementation**

## **Data Split**

In [55]:
# Predictor & Target Data Split
x = df['text_stemmed'].values
y = df['target'].values

# Train & Test Data Split
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size=0.2,
                                                #   stratify=Y,
                                                  random_state=2)

print('Train Data Shape: ',x_train.shape,y_train.shape)
print('Test Data Shape: ',x_test.shape,y_test.shape)

Train Data Shape:  (1280000,) (1280000,)
Test Data Shape:  (320000,) (320000,)


## **Vectorization**

In [57]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [67]:
# print(x_train)

In [68]:
# print(x_test)

## **Model 01 - Logistic Regression**

In [81]:
model1 = LogisticRegression(max_iter=1000)
model1.fit(x_train,y_train)

# Training Accuracy
y_train_pred = model1.predict(x_train)
model1_train_accuracy = accuracy_score(y_train,y_train_pred)
print('Training Data Accuracy: ',model1_train_accuracy)

# Testing Accuracy
y_test_pred = model1.predict(x_test)
model1_test_accuracy = accuracy_score(y_test,y_test_pred)
print('Test Data Accuracy: ',model1_test_accuracy)

Training Data Accuracy:  0.81006796875
Test Data Accuracy:  0.7782375


In [82]:
model1.coef_

array([[-0.92257935, -1.20661965, -0.23264002, ...,  0.11698342,
        -0.13649419, -0.06928259]])

## **Exporting Trained Model**

In [78]:
import pickle

model1_file = open('Model01.pkl','wb')
pickle.dump(model1,model1_file)

## **Importing Trained Model**

In [79]:
# del(model1)
# model1 = pickle.load(open('Model01.pkl','rb'))

EOFError: Ran out of input